In [14]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

from sklearn.model_selection import train_test_split 

In [15]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'BOG'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=20)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 1

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [16]:
clear_buyers = get_fresh_query_result("https://internal-redash.federate.frubana.com/",136016,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [17]:
clear_buyers["last_order_date"] = pd.to_datetime(clear_buyers["last_order_date"]) 
clear_buyers = clear_buyers[clear_buyers.last_order_date >= customer_date_desde].copy()
clear_buyers.head()

,customer_id,last_order_date,recency_days,registered_city
0,308238668,2023-02-28,0,BAQ
1,307548193,2023-02-27,1,SPO
2,307574923,2023-02-27,1,CWB
3,307577156,2023-02-27,1,BOG
4,307508893,2023-02-27,1,BAQ


In [18]:
query = """
WITH
info AS (
SELECT DISTINCT
    s.site_identifier_value as region_code,
    --ffg.close_date,
    bo.customer_id,
    --bs.addl_product_id as padre_sku_id,
    --bs2.name AS card_name,
    --bcat2.name AS category,
    bcat.name AS subcat,
    COUNT(DISTINCT EXTRACT(WEEK FROM ffg.close_date)) AS count_weeks,
    --baid.adjustment_reason,
    SUM ( (boi.quantity * foi.step_unit * boi.sale_price)/{tipo_cambio}) AS gmv_usd
    --SUM ( COALESCE((baid.adjustment_value * boip.quantity * foi.step_unit)/{tipo_cambio}, 0) ) AS discount_applied,
    --SUM ( boi.quantity* foi.step_unit ) AS cant

FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"     bfgi
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg        ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg        ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                bo         ON bo.order_id = bfg.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s          ON bo.site_disc = s.site_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"           boi        ON bfgi.order_item_id= boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"            foi        ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo         ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo        ON bfo.fulfillment_group_id= bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"  bdoi       ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs         ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"              bp         ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                  bs2        ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"             bcat       ON bcat.category_id = bp.default_category_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop        ON bop.order_id = bo.order_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot        ON fot.fb_order_type_id=fo.fb_order_type_id
LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_category_xref"        bcx        ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"              bcat2      ON bcx.category_id = bcat2.category_id
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"  boip       ON  boi.order_item_id = boip.order_item_id
--LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"    baid       ON  baid.order_item_price_dtl_id = boip.order_item_price_dtl_id

WHERE 
   DATE(ffg.close_date) >= current_date - interval '10 week' 
   AND DATE(ffg.close_date) <=  current_date
   AND bo.order_status='SUBMITTED'
   AND fo.fb_order_status_id IN (1,6,7,8) 
   AND s.site_identifier_value = '{ciudad}'
   AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
   AND bop.archived = 'N'
   AND (fot.name IS NULL OR fot.name <> 'REFUND')
   -- FIX SUPER DESCUENTOS
   AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
   AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'

GROUP BY 1,2,3--,5,6,7
)

SELECT 
*
FROM info
WHERE count_weeks > 5
""".format(tipo_cambio=t_cambio, ciudad=city_code)

data_ventas_aux = read_connection_data_warehouse.runQuery(query)
data_ventas_aux[["gmv_usd"]] = data_ventas_aux[["gmv_usd"]].astype(float)
#data_ventas["submit_date"] = pd.to_datetime(data_ventas["submit_date"])

In [6]:
data_ventas_aux.head()

,region_code,customer_id,subcat,count_weeks,gmv_usd
0,BOG,-212085,Verduras,10,354.683525
1,BOG,35022844,Pulpas de Fruta,6,81.118113
2,BOG,19776702,Huevos,8,263.691472
3,BOG,56376131,Papel & Toallas,6,10.987022
4,BOG,225278759,Verduras,6,119.325608


In [7]:
if city_code == 'BOG':
    subcats = ['Azúcar & Endulzantes',  'Aceites & Grasas',  'Arroz',  'Harinas & Mezclas',  'Granos',  'Detergente, Jabón & Lavalozas',  'Papel & Toallas',
            'Desinfectantes & Sanitizantes',  'Cervezas & Licores',  'Pollo Fresco',  'Res Fresco',  'Res Congelado',  'Cerdo Fresco',  'Pescados & Mariscos Congelado', 
            'Alimentos Congelados',  'Contenedores',  'Huevos']
elif city_code == 'BAQ':
    subcats = ['Aceites & Grasas','Alimentos Congelados','Arroz','Azúcar & Endulzantes','Contenedores',
               'Gaseosas','Harinas & Mezclas','Huevos','Pollo Congelado','Tubérculos']
elif city_code == 'SPO':
    subcats = ['Cervejas', 'Refrigerantes e Energéticos', 'Suínos', 'Carne Vermelha', 'Frango', 'Peixes e Frutos do Mar', 'Embutidos', 
            'Batatas Pré-Fritas', 'Legumes', 'Leite', 'Ovos', 'Queijos', 'Detergente, Sabão e Lava-Louças', 'Arroz', 'Café, Chocolates e Infusões', 
            'Feijão', 'Massas e Molhos', 'Azeites, Óleos e Vinagres', 'Frutas']
else:
    print("city does not exist yet")

In [8]:
grouped = data_ventas_aux.groupby('subcat').gmv_usd.sum().reset_index()
grouped['weight'] = grouped.gmv_usd / grouped.gmv_usd.sum()
grouped = grouped.sort_values(by=['weight'], ascending=False)
grouped['Cumulative_Weight'] = grouped['weight'].cumsum()
grouped = grouped[(grouped.Cumulative_Weight <= 0.8) & (grouped.subcat.isin(subcats))] 
subcats_exp = grouped.subcat.unique()

In [9]:
data_ventas = data_ventas_aux[data_ventas_aux.subcat.isin(subcats_exp)].copy()

In [10]:
f = data_ventas.groupby("subcat", as_index=False).agg({'customer_id':[lambda x: x.nunique(),lambda x: list(x.unique())]}).droplevel(0, axis="columns")
f.columns = ['subcat', 'customers', 'list_customers']
f

,subcat,customers,list_customers
0,Aceites & Grasas,1503,"[142238649, 43094225, 154119311, 21795104, 211..."
1,Arroz,1710,"[44220547, 285665337, 44592021, 61412609, 2101..."
2,Azúcar & Endulzantes,1181,"[-240842, 208387924, 23015960, 85844101, 40639..."
3,Contenedores,1469,"[50555161, 204317012, 44697565, 68803684, 4590..."
4,Harinas & Mezclas,776,"[227706949, 106346093, 21349844, 15329618, 563..."
5,Huevos,793,"[19776702, 209940589, 43119904, 175268690, 526..."


In [11]:
def create_groups(subcat):
    s = f[f.subcat == subcat].list_customers.to_numpy()[0]
    
    d = data_ventas[(data_ventas.customer_id.isin(s)) & (data_ventas.subcat == subcat)].groupby("customer_id").gmv_usd.mean().reset_index()
    
    customer_control, customer_test = train_test_split(s, test_size = 0.50)
    
    alpha=0.01
    aux=0
    
    while abs(1-(d[d.customer_id.isin(customer_control)]['gmv_usd'].mean()/d[d.customer_id.isin(customer_test)]['gmv_usd'].mean())) > alpha:
        customer_control, customer_test = train_test_split(s, test_size = 0.50)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
        
    all_users = clear_buyers[(~clear_buyers.customer_id.isin(customer_test)) & (clear_buyers.registered_city == city_code)].customer_id.unique().tolist()
    customer_control = [int(customer) for customer in customer_control]
    customer_test = [int(customer) for customer in customer_test]
    
    segt_offer = {'name':"HOOK_Elasticidad"+todays_date+"_"+city_code+"_"+subcat+"_ALL_F1", 'customersIds': list(all_users)}
    segt_control = {'name':"HOOK_Elasticidad"+todays_date+"_"+city_code+"_"+subcat+"_CONTROL_F1", 'customersIds': list(customer_control)}
    segt_test = {'name':"HOOK_Elasticidad"+todays_date+"_"+city_code+"_"+subcat+"_TEST_F1", 'customersIds': list(customer_test)}
    
    print("SUBCAT:",subcat, "// TOTAL_USERS:", len(all_users), "// TEST:",len(customer_test), "// CONTROL:",len(customer_control))
    
    return customer_control, customer_test, segt_test, segt_control, segt_offer

In [12]:
global_segment_list = []
r= {}

for subcat in f.subcat:
    control, test, segment_t, segment_c, segment_o = create_groups(subcat)
    
    #r['CONTROL_'+todays_date+"_"+city_code+"_"+subcat] = control
    #r['TEST_'+todays_date+"_"+city_code+"_"+subcat] = test
    
    global_segment_list.append(segment_t)
    global_segment_list.append(segment_c)
    global_segment_list.append(segment_o)

df_test_control = pd.DataFrame.from_dict(r, orient='index').T

SUBCAT: Arroz // TOTAL_USERS: 15689 // TEST: 1075 // CONTROL: 1074
SUBCAT: Azeites, Óleos e Vinagres // TOTAL_USERS: 15200 // TEST: 1713 // CONTROL: 1713
SUBCAT: Cervejas // TOTAL_USERS: 16204 // TEST: 354 // CONTROL: 354
SUBCAT: Feijão // TOTAL_USERS: 15929 // TEST: 814 // CONTROL: 813
SUBCAT: Frutas // TOTAL_USERS: 14732 // TEST: 2244 // CONTROL: 2243
SUBCAT: Legumes // TOTAL_USERS: 14153 // TEST: 2907 // CONTROL: 2906
SUBCAT: Leite // TOTAL_USERS: 15927 // TEST: 780 // CONTROL: 779
SUBCAT: Queijos // TOTAL_USERS: 16085 // TEST: 537 // CONTROL: 537
SUBCAT: Refrigerantes e Energéticos // TOTAL_USERS: 15987 // TEST: 660 // CONTROL: 660


In [13]:
len(clear_buyers[clear_buyers.registered_city == city_code])

16493

In [14]:
def crear_seg(global_value):
  import requests
  import json

  url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

  payload = json.dumps(global_value)


  headers = {
    'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)
  return response.text

In [15]:
sgt = []
i=0
for global_seg in global_segment_list:
    rep_text = crear_seg([global_seg])
    sgt.append((rep_text[32:].strip('][')[:6], rep_text[32:].strip('][')[38:]))
    #i+=1
    #if i == 3:
    #    break
    time.sleep(20)
sgt

[('123406', 'Arroz_TEST_F1'),
 ('123951', 'Arroz_CONTROL_F1'),
 ('123952', 'Arroz_ALL_F1'),
 ('123953', 'Azeites, Óleos e Vinagres_TEST_F1'),
 ('123553', 'Azeites, Óleos e Vinagres_CONTROL_F1'),
 ('123852', 'Azeites, Óleos e Vinagres_ALL_F1'),
 ('123853', 'Cervejas_TEST_F1'),
 ('123904', 'Cervejas_CONTROL_F1'),
 ('123804', 'Cervejas_ALL_F1'),
 ('123905', 'Feijão_TEST_F1'),
 ('123954', 'Feijão_CONTROL_F1'),
 ('123906', 'Feijão_ALL_F1'),
 ('123907', 'Frutas_TEST_F1'),
 ('123705', 'Frutas_CONTROL_F1'),
 ('123657', 'Frutas_ALL_F1'),
 ('123606', 'Legumes_TEST_F1'),
 ('123955', 'Legumes_CONTROL_F1'),
 ('123908', 'Legumes_ALL_F1'),
 ('123607', 'Leite_TEST_F1'),
 ('123608', 'Leite_CONTROL_F1'),
 ('123805', 'Leite_ALL_F1'),
 ('123554', 'Queijos_TEST_F1'),
 ('123555', 'Queijos_CONTROL_F1'),
 ('123706', 'Queijos_ALL_F1'),
 ('123609', 'Refrigerantes e Energéticos_TEST_F1'),
 ('123854', 'Refrigerantes e Energéticos_CONTROL_F1'),
 ('123806', 'Refrigerantes e Energéticos_ALL_F1')]

In [16]:
sgt[0][1]

'Arroz_TEST_F1'

In [17]:
segmentos = [i[0] for i in sgt if i[1][i[1].index("_")+1:i[1].index("_", i[1].index("_")+1)] == 'ALL']
subcategorias = [i[1][:i[1].index("_")] for i in sgt if i[1][i[1].index("_")+1:i[1].index("_", i[1].index("_")+1)] == 'ALL']

In [18]:
for i,j in enumerate(segmentos):
    print(str(j)+"\t"+subcategorias[i])

123952	Arroz
123852	Azeites, Óleos e Vinagres
123804	Cervejas
123906	Feijão
123657	Frutas
123908	Legumes
123805	Leite
123706	Queijos
123806	Refrigerantes e Energéticos
